In [1]:
import torch
import numpy as np
from torchvision import transforms
import torchvision
from torch.utils.tensorboard import SummaryWriter
import datasets as mt_datasets
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
import imageio

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

writer = SummaryWriter()

nodules = torch.load('nodules.pt')
malignancies = torch.load('malignancies.pt') - 1

In [3]:
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

In [4]:
dataset = mt_datasets.TensorDataset(nodules, malignancies)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [5]:
print(len(val_indices))

102


In [6]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


In [7]:
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 5)
resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)


In [ ]:
epochs = 1
idx = 0
for epoch in range(epochs):
    for i, (nodule, malignancy) in enumerate(train_loader, 0):
        nodule, malignancy = nodule.to(device), malignancy.to(device)


        # get the inputs; data is a list of [inputs, labels]
        input = torch.cat([nodule.mean(dim=-1).unsqueeze(1), nodule.mean(dim=-2).unsqueeze(1), nodule.mean(dim=-3).unsqueeze(1)], dim=1)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(input)
        loss = criterion(outputs, malignancy)
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Loss/train', loss.item(), idx)
        print(f"Epochs: {epoch}, i = {i}, Loss = {loss.item()}")
        idx += 1

print('Finished Training')

Epochs: 0, i = 0, Loss = 1.777927041053772
Epochs: 0, i = 1, Loss = 1.6741379499435425
Epochs: 0, i = 2, Loss = 1.482314109802246
Epochs: 0, i = 3, Loss = 2.0598957538604736
Epochs: 0, i = 4, Loss = 1.3544490337371826
Epochs: 0, i = 5, Loss = 1.2030029296875
Epochs: 0, i = 6, Loss = 2.13533091545105
Epochs: 0, i = 7, Loss = 2.137643337249756
Epochs: 0, i = 8, Loss = 1.4781906604766846
Epochs: 0, i = 9, Loss = 2.511857509613037
Epochs: 0, i = 10, Loss = 1.1556068658828735
Epochs: 0, i = 11, Loss = 1.5826786756515503
Epochs: 0, i = 12, Loss = 1.8691275119781494
Epochs: 0, i = 13, Loss = 1.461388349533081
Epochs: 0, i = 14, Loss = 1.7714548110961914
Epochs: 0, i = 15, Loss = 1.3291492462158203
Epochs: 0, i = 16, Loss = 1.337815761566162


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        images, labels = data
        outputs = resnet18(images)
        print(len(outputs))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))